<a href="https://colab.research.google.com/github/Anshika-Choudhary/CPI/blob/main/Consumer_Price_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL NECESSARY LIBRARIES



In [3]:
pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 43.3 MB/s eta 0:00:00


IMPORT LIBRARIES

In [4]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np

<ipython-input-4-cd0b8341c3e1>:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-4-cd0b8341c3e1>:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


IMPORT THE DATASET

In [6]:
data = pd.read_csv('data(saharanpur).csv')

DASHBOARD LAYOUT

In [7]:

# convert the date column to a datetime object
data['date'] = pd.to_datetime(data['date'])

# create a new column of month using apply and lambda
data['Month'] = data['date'].apply(lambda x: x.month)
data['Year'] = data['date'].apply(lambda x: x.year)
print(data)

           id       date  commodity_group  commodity_id  \
0     6092770 2022-01-01  Grains & Pulses             1   
1     6092898 2022-01-01  Grains & Pulses             2   
2     6093000 2022-01-01      Edible Oils             3   
3     6093121 2022-01-01           Others             4   
4     6093249 2022-01-01  Grains & Pulses             5   
...       ...        ...              ...           ...   
5077  7154658 2022-10-31       Vegetables            18   
5078  7154885 2022-10-31  Grains & Pulses            19   
5079  7155350 2022-10-31      Edible Oils            21   
5080  7155118 2022-10-31  Grains & Pulses            20   
5081  7155564 2022-10-31  Grains & Pulses            22   

                   commodity        zone  centre_id      centre  price  Month  \
0               Atta (Wheat)  North Zone        144  Saharanpur     34      1   
1                   Gram Dal  North Zone        144  Saharanpur     72      1   
2     Groundnut Oil (Packed)  North Zone        

In [8]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1('Retail Price and CPI Dashboard'),
    html.Div([
        html.Label('Select District:'),
        dcc.Dropdown(
            id='district-dropdown',
            options=[{'label': i, 'value': i} for i in data['centre'].unique()],
            value=data['centre'].iloc[0]
        )
    ]),
    html.Div([
        html.Label('Select Commodity:'),
        dcc.Dropdown(
            id='commodity-dropdown',
            options=[{'label': i, 'value': i} for i in data['commodity'].unique()],
            value=data['commodity'].iloc[0]
        )
    ]),
    dcc.Graph(id='price-graph'),
    html.Div(id='cpi-output')
])


In [9]:
@app.callback(
    Output('price-graph', 'figure'),
    Input('district-dropdown', 'value'),
    Input('commodity-dropdown', 'value')
)
def update_price_graph(district, commodity):
    filtered_data = data[(data['centre'] == district) & (data['commodity'] == commodity)]
    x = filtered_data['Month'].unique()
    y = filtered_data.groupby('Month')['price'].mean().values
    fig = {
        'data': [{
            'x': x,
            'y': y,
            'type': 'line'
        }],
        'layout': {
            'title': 'Monthly Change in Retail Price',
            'xaxis': {'title': 'Month'},
            'yaxis': {'title': 'Retail Price'}
        }
    }
    return fig





In [10]:
!pip install pyngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=77fe9626dc200da93d8c168787cf02f4e8ed80e492b764e7d5dd3241409108b5
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [11]:
from pyngrok import ngrok
public_url = ngrok.connect(port='8050')
print(public_url)


NgrokTunnel: "http://e33d-35-229-42-248.ngrok.io" -> "http://localhost:80"


In [ ]:
@app.callback(
    Output('cpi-output', 'children'),
    Input('district-dropdown', 'value')
)


In [ ]:
def calculate_cpi(district):
    filtered_data = data[data['centre'] == district]
    start_date = pd.to_datetime(filtered_data['date']).min()
    end_date = pd.to_datetime(filtered_data['date']).max()
    cpi_data = pd.DataFrame({'Date': pd.date_range(start_date, end_date, freq='M')})
    cpi_data['CPI'] = cpi_data['Date'].apply(lambda x: np.average(filtered_data[filtered_data['Date'].str.startswith(str(x.year))]['CPI']))
    cpi = np.average(cpi_data['CPI'])
    return f'CPI of {district} for the year is {cpi:.2f}'